# Formation ML Flow

L'objectif de cette formation est de vous apprendre à utiliser ML Flow, un dashboard pour expériences de Machine Learning. Cette formation vous montrera pas à pas comment lancer une expérience de Machine Learning et surveiller les paramètres, métriques et resultats de votre expérience grâce à ML Flow. 

Pour cela, la formation s'appuiera sur le célèbre jeu de donnée Iris, qui classifie 150 echantillons de fleur en 3 espèces d'iris : Iris setosa, Iris versicolor et Iris virginica. Le dataset comprend les caractéristiques de chaque échantillon (longueur et largeur des pétales et longueur et largeur des sépales) et nous allons donc essayer de créer plusieurs modèles qui essaieront de classifier les échantillons d'iris selon leurs caractéristiques. 

Commençons par importer les librairies qui vont nous servir :

In [ ]:
import mlflow
import numpy as np 
import matplotlib.pyplot as plt 
import pandas as pd 
import time
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

# 1. Ouverture et visualisation du dataset Iris

A présent ouvrons le fichier contenant le dataset et affichons les données concernant les 5 premiers échantillons :

In [ ]:
dataset = pd.read_csv('iris.csv')
dataset.head()

Affichons maintenant quelques informations sur les caractéristiques des fleurs :

In [ ]:
dataset.describe()

Affichons le nombre d'échantillons pour chacune des espèces d'iris :

In [ ]:
dataset.groupby('class').size()

Affichons maintenant le nuage de points de chaque paire de caractéristiques. Sur la diagonale, on retrouve un histogramme du nombre d'échantillons selon la valeur de la caractéristique. On peut remarquer certaines structures, par exemple des groupes, dans les relations entre certaines caractéristiques.

In [ ]:
pd.plotting.scatter_matrix(dataset,figsize=(15,7))
plt.suptitle('Scatter matrix of the Iris dataset')
plt.show()

# 2. ML Flow : premiers pas avec un modèle aléatoire

Nous allons maintenant réaliser un premier exemple de modèle de classification très simple : le modèle aléatoire. Ce modèle attribuera aléatoirement une classe à chaque échantillon du dataset. Nous visualiserons ensuite ses performances (pas très bonnes, on l'imagine) avec ML FLow.

Commençons par séparer le dataset en 2 variables : les 4 caractéristiques d'une part (X), et l'espèce d'iris d'autre part (Y).

In [ ]:
array = dataset.values
X = array[:,0:4]
Y = array[:,4]

Construisons notre modèle aléatoire :

In [ ]:
from sklearn.dummy import DummyClassifier
dummy = DummyClassifier(strategy="uniform")
dummy.fit(X, Y)

Affichons ses prédictions pour les premiers échantillons :

In [ ]:
Y_pred = dummy.predict(X)
for i in range(5):
    print(X[i], ', vraie classe : ', Y[i], ', prédiction : ', Y_pred[i])

Nous allons maintenant enregistrer les performances de ce magnifique modèle avec ML Flow. Pour l'instant executez les cellules suivantes "bêtement", nous expliquerons chaque ligne ensuite.

In [ ]:
with mlflow.start_run(run_name='Modele_Aleatoire'):
    mlflow.log_param('MODEL_NAME', 'DummyClassifier')
    mlflow.log_metric('Accuracy', accuracy_score(Y, Y_pred))
    mlflow.log_artifact('iris.csv')

Maintenant ouvrez un terminal et naviguez jusqu'au dossier ```formation-mlflow```. Affichez le contenu du dossier. Vous devriez y trouver un nouveau dossier nommé ```mlflow```. Ce dossier contient toutes les informations que nous avons enregistré précédemment.

Toujours dans le dossier ```formation-mlflow```, entrez la commande ```mlflow ui```. Le message ```Serving on http://XXXXXXX:5000``` devrait s'afficher. Ouvrez maintenant un navigateur web et ouvrez une fenêtre vers l'url ```http://localhost:5000/```. Vous devriez voir apparaître le dashboard suivant :

![dashboard](MLFlow.png)

La ligne au milieu du tableau, comportant le nom ```Modele_Aleatoire``` correspond aux données que nous avons enregistré précédemment. Vous pouvez remarquer dans la colonne ```Parameters``` le paramètre  ```MODEL_NAME``` que nous avons enregistré, ainsi que la métrique ```Accuracy``` dans la colonne ```Metrics```. Maintenant cliquez sur cette ligne et vous trouverez le détail des informations concernant ce ```run```. Descendez jusqu'à la section ```Artifacts``` où vous pourrez voir une copie du fichier ```iris.csv```. En cliquant dessus vous pourrez le visualiser en entier.

Revenons maintenant sur la façon dont nous avons enregistré ces informations avec ML FLow.

ML Flow permet d'enregistrer puis visualiser 3 types de choses :
* des **paramètres** : ce sont des valeurs (int, float, string, ...) qui ne varient pas au cours d'un 'run', ici ```MODEL_NAME``` est un paramètre.
* des **métriques** : ce sont des valeurs numériques qui peuvent varier au cours du 'run', ici ```Accuracy``` est une métrique.
* des **fichiers** : ces fichiers peuvent prendre n'importe quelle forme (png, jpeg, gif, txt, ...) et ne sont pas modifiables au cours du 'run. Ici nous avons fait une copie du fichier ```iris.csv```.

Toutes ces variables sont regroupées dans un même 'run', ici nommé 'Modele_Aleatoire'. Un 'run' correspond à une ligne du tableau. Les 'runs' peuvent être regroupés en 'experiences', ici nommée ```Default``` dans la colonne de gauche. Nous verons comment changer le nom de l'expérience par la suite.

Reprenons les lignes de code qui nous ont permis de faire tout ça :

In [ ]:
# 1. Création du 'run' ML FLow, nommé ici 'Modele_Aleatoire'
#    Tous les paramètres, métriques ou fichiers que nous enregistrerons ensuite
#    seront enregistré dans ce 'run', qui correspond à une ligne du tableau
with mlflow.start_run(run_name='Modele_Aleatoire'):

    # 2. Enregistrement d'un paramètre
    mlflow.log_param('MODEL_NAME', 'DummyClassifier')

    # 3. Enregistrement d'une métrique
    mlflow.log_metric('Accuracy', accuracy_score(Y, Y_pred))

    # 4. Enregistrement d'un fichier
    mlflow.log_artifact('iris.csv')

## Exercice d'application

En traçant l'histogramme des échantillons selon la valeur de ```petal-width```, on voit qu'on peut assez facilement discriminer les échantillons selon cette variable.

In [ ]:
dataset.groupby('class')['petal-width'].plot.hist()
plt.xlabel('petal-width')
plt.title('Histogramme des échantillons selon la valeur de petal-width')
plt.show()

 Nous allons donc essayer de coder notre propre modèle de classification selon cette spécificité et enregistrer ses performances avec ML FLow.

Pour cela, commencez par coder une fonction qui :
* prend en entrée une liste d'échantillons
* retourne la liste des classes des échantillons
* attribue la classe selon la valeur de ```petal-width``` (dernière colonne de l'échantillon). On pourra par exemple déterminer la classe en comparant ```petal-width``` à des seuils bien choisis.

In [ ]:
# %load solutions/solution1
# Si vous êtes bloqués, décommentez la ligne précédente et exécutez là pour afficher une solution

def simple_classification_model(list_samples):
    list_classes = []

    # AJOUTEZ VOTRE CODE ICI

    return list_classes

Vérifions ici que notre modele fonctionne sur quelques échantillons :

In [ ]:
Y_pred = simple_classification_model(X)
for i in range(5):
    print(X[i], ', vraie classe : ', Y[i], ', prédiction : ', Y_pred[i])

Maintenant à vous d'utiliser MLFLow pour enregistrer l'accuracy de ce nouveau modèle !

In [ ]:
# %load solutions/solution2
# Si vous êtes bloqués, décommentez la ligne précédente et exécutez là pour afficher une solution

# AJOUTER VOTRE CODE ICI

# 3. ML Flow : Comparaison de plusieurs modèles

Nous allons maintenant entraîner différents modèles sur le jeu de données et voir comment nous pouvons les comparer facilement avec ML FLow. Pour cela, nous allons créer plusieurs modèles et estimer leurs performances sur des données qu'ils n'ont pas vu pendant l'entraînement. Commençons par séparer notre dataset en deux parties : une pour l'entraînement et une pour le test.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.20, random_state=1)

Construisons la liste des modèles à tester :

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

models = []
models.append(('LogisticRegression', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LinearDiscriminantAnalysis', LinearDiscriminantAnalysis()))
models.append(('KNeighborsClassifier', KNeighborsClassifier()))
models.append(('DecisionTreeClassifier', DecisionTreeClassifier()))
models.append(('GaussianNB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))

Ajoutons cette fonction qui va nous permettre de tracer de jolies matrices de confusion par la suite :

In [ ]:
def plot_confusion_matrix(cm, target_names, title, 
                          normalize=True, save_path='matrix.png'):
    import itertools
    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy
    if normalize: cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    cmap = plt.get_cmap('Blues')
    plt.figure(figsize=(5,5))
    plt.imshow(cm, interpolation='nearest', cmap=cmap, vmin=0, vmax=1)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(target_names))
    plt.xticks(tick_marks, target_names, rotation=45)
    plt.yticks(tick_marks, target_names)

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label\naccuracy={:0.4f}; misclass={:0.4f}'\
               .format(accuracy, misclass))
    plt.gcf().canvas.draw()
    plt.savefig(save_path)
    plt.close()

Nous allons maintenant entraîner nos modèles et enregistrer leurs paramètres et métriques avec ML Flow. 

Nous allons voir deux nouvelles choses : 
* Comment créer une nouvelle expérience
* Comment enregistrer plusieurs paramètres ou métriques d'un seul coup

In [ ]:
# 1. Création d'une nouvelle expérience
#    Tous les nouveaux runs seront enregistrés dans cette expérience
mlflow.set_experiment('Comparaison')

for name, model in models:
    print(name)

    # 2. Création et début d'un nouveau run
    with mlflow.start_run(run_name=name):

        # 3. Enregistrement de plusieurs paramètres sous forme d'un dictionnaire
        params = {}
        params['MODEL_NAME'] = name
        params['TRAIN_SIZE'] = len(X_train)
        params['TEST_SIZE'] = len(X_val)
        mlflow.log_params(params)

        # On note le moment du début de l'expérience pour mesurer la durée de l'entraînement
        start = time.time()

        # Entraînement du modèle
        model.fit(X_train, Y_train)

        training_time = time.time() - start

        predictions = model.predict(X_val)

        # 4. Enregistrement de plusieurs métriques sous forme d'un dictionnaire
        metrics = {}
        metrics['Accuracy'] = accuracy_score(Y_val, predictions)
        metrics['Precision'] = precision_score(Y_val, predictions, average='macro')
        metrics['Recall'] = recall_score(Y_val, predictions, average='macro')
        metrics['Training Time'] = training_time
        mlflow.log_metrics(metrics)

        # 5. Enregistrement de la matrice de confusion
        cm = confusion_matrix(Y_val, predictions)
        plot_confusion_matrix(cm, ['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'],
                              'Confusion matrix '+name)
        mlflow.log_artifact('matrix.png')

Retournez maintenant sur votre navigateur pour voir le résultat dans le dashboard ML Flow.

Vous devriez voir une nouvelle expérience 'Comparaison' dans la liste des expériences à gauche. En cliquant dessus vous pourrez voir apparaître les 6 runs que nous venons de lancer. Selectionnez les tous et cliquez sur ```Compare```. Vous pouvez maintenant facilement comparer toutes les caractéristiques de nos modèles. En cliquant sur le nom d'une des métriques vous pourrez afficher un diagramme comparant les valeurs pour tous les modèles.

Revenez au tableau des runs et ouvrer l'un des runs. Dans la partie ```Artifacts``` vous pourrez voir la superbe matrice de confusion que nous avons tracé.

## Exercice d'application

Nous pouvons maintenant comparer le modèle que nous avions codé précedemment dans la fonction ```simple_classification_model``` avec les autres modèles, en enregistrant tout cela dans MLFlow. A vous de jouer !

In [ ]:
# %load solutions/solution3
# Si vous êtes bloqués, décommentez la ligne précédente et exécutez là pour afficher une solution

# AJOUTEZ VOTRE CODE ICI

# 4. ML Flow : Enregistrer l'évolution d'une métrique

Dans cette dernière partie, nous allons voir comment enregistrer l'évolution d'une métrique au cours du temps. Pour cela, nous allons utiliser comme modèle un réseau de neurones basique : le perceptron multi-couches.

Afin de calculer nos métriques au cours de l'entrainement, nous decoupons celui-ci en plusieurs epochs. A chaque epoch, le reseau de neurone s'entraîne une fois sur l'intégralité du dataset. Tout ceci est fait automatiquement dans la fonction ```fit``` par défaut de scikit learn, mais nous allons faire les choses manuellement ici, pour visualiser l'évolution des métriques au cours de l'apprentissage.

In [ ]:
from sklearn.neural_network import MLPClassifier
import time

# 1. Création et début d'un nouveau run
with mlflow.start_run(run_name='Multilayer Perceptron'):

    model = MLPClassifier(max_iter=10)
    mlflow.log_param('MODEL_NAME', 'MLPClassifier')

    N_TRAIN_SAMPLES = X_train.shape[0]
    N_EPOCHS = 50
    N_BATCH = 8
    N_CLASSES = np.unique(Y_train)
    scores_train = []
    scores_test = []

    # EPOCH
    epoch = 0
    while epoch < N_EPOCHS:
        if epoch % 5 == 0 : print('Epoch: ', epoch)
        # SHUFFLING
        random_perm = np.random.permutation(X_train.shape[0])
        mini_batch_index = 0
        while True:
            # TRAIN ON MINI-BATCH
            indices = random_perm[mini_batch_index:mini_batch_index + N_BATCH]
            model.partial_fit(X_train[indices], Y_train[indices], classes=N_CLASSES)
            mini_batch_index += N_BATCH
            if mini_batch_index >= N_TRAIN_SAMPLES:
                break

        metrics = {}
        predictions = model.predict(X_val)
        metrics['Accuracy'] = accuracy_score(Y_val, predictions)
        metrics['Precision'] = precision_score(Y_val, predictions, average='macro')
        metrics['Recall'] = recall_score(Y_val, predictions, average='macro')
        metrics['Loss'] = model.loss_
        # 2. Enregistrement de plusieurs métriques pour une epoch donnée
        mlflow.log_metrics(metrics, step=epoch)
        time.sleep(3)

        epoch += 1

Revenez sur le tableau des runs dans le dashboard et ouvrez le run que nous venons de créer. En cliquant sur l'une des métriques, vous pourrez voir son évolution au cours de l'apprentissage du réseau de neurones.

# 5. Un exercice d'application météo

Pour finir, je vous propose un exercice d'application sur le thème de la météo, qui regroupera tout ce que nous avons appris ici.

Pour cela nous allons étudier la température sur l'île d'Ouessant : le fichier ```ouessant_temperature.csv``` contient l'historique des températures à Ouessant, mesurées toutes les 6 minutes durant l'année 2018.



In [ ]:
meteo_dataset = pd.read_csv('ouessant_temperature.csv')

In [ ]:
meteo_dataset.t.plot(figsize=(15,7))
plt.title("Température sur l'île d'Ouessant au cours de l'année 2018")
plt.ylabel('Température (K)')
plt.xlabel('Mois')
plt.xlim(0,87453)
plt.xticks(np.arange(0,87453,7286), ['J','F','M','A','M','J','J','A','S','O','N','D'])
plt.grid(axis='both')
plt.show()

Notre objectif est de comparer des modèles de prévision de température. Ces modèles prendront en entrée l'historique des températures sur 2h et devront prévoir la température pendant l'heure suivante.

Commençons par créer notre dataset de séquences de température sur 3h.

In [ ]:
data = meteo_dataset.t.values
len_sequence = 30   # 10 samples per hour during 3 hours
dataset = []
i = 0
while i < len(data)-len_sequence:
    dataset.append(data[i:i+len_sequence])
    i += len_sequence
dataset = np.stack(dataset, axis=0)
dataset.shape

Séparons les données en X et Y, puis créons un jeu d'entraînement et un jeu de test :

In [ ]:
X = dataset[:,:20]
Y = dataset[:,20:]
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.20, random_state=1)

Maintenant, à vous de jouer ! 
* Créez au moins **3 modèles de prévisions** de températures comme nous l'avons fait dans le premier exercice. On pourra commencer par des modèles simples tels que la persistance.
* Ajoutez au moins **3 métriques** pour évaluer les performances de vos modèles.
* Pour chaque métrique, évaluez là sur l'ensemble de votre prévision, mais aussi sur chacune des échéances (t1, t2, t3,...) afin de tracer dans MLFlow la courbe d'évolution de cette métrique.
* Enregistrez tout cela dans MLFlow, dans une nouvelle expérience.

In [ ]:
# AJOUTEZ VOTRE CODE ICI